In [1]:
import os

import pymysql
import pandas as pd

from sqlalchemy import create_engine
from dotenv import load_dotenv


In [2]:
# setup env
load_dotenv()

True

In [3]:
# setup db
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

conn_text = 'mysql+pymysql://{}:{}@{}/{}'.format(user,password,host,db)
sqlEngine = create_engine(conn_text)

In [4]:
# load data from db
dbConnection = sqlEngine.connect()
tag = pd.read_sql("select * from basic_tag", dbConnection);
event_tags = pd.read_sql("select * from basic_event_tags", dbConnection);
tag_category = pd.read_sql("select * from basic_tagcategory", dbConnection);
event = pd.read_sql("select * from basic_event", dbConnection);
experiment = pd.read_sql("select * from basic_experiment", dbConnection);
experiment_item = pd.read_sql("select * from basic_experimentitem", dbConnection);
request_log = pd.read_sql("select * from rest_framework_tracking_apirequestlog LIMIT 1000000;", dbConnection);
event_material_list = pd.read_sql("select * from basic_event_material_list", dbConnection);

dbConnection.close()

In [5]:
# transform request_log data
request_log = request_log[['requested_at', 'path', 'query_params', 'status_code']]

In [6]:
# transform event data
event = event[event['isActive'] == True]
event = event[['id', 'title', 'description', 'costsRating', 'executionTimeRating', 'isPrepairationNeeded', 'imageLink', 'created_at', 'createdBy']]


In [7]:

event.to_parquet('data/event.parquet', coerce_timestamps="us")
tag.to_parquet('data/tag.parquet', coerce_timestamps="us")
event_tags.to_parquet('data/event_tags.parquet', coerce_timestamps="us")
tag_category.to_parquet('data/tag_category.parquet', coerce_timestamps="us")
experiment.to_parquet('data/experiment.parquet', coerce_timestamps="us")
request_log.to_parquet('data/request_log.parquet', coerce_timestamps="us")
experiment_item.to_parquet('data/experiment_item.parquet', coerce_timestamps="us")
event_material_list.to_parquet('data/event_material_list.parquet', coerce_timestamps="us")